# Cost Surface Analysis Operations

In [6]:
import arcpy
import requests
import io
import zipfile

## Display Dory's Farm and North Picnic Area in ArcGIS Pro from CSV Table to XY to Point Tool

In [2]:
arcpy.management.XYTableToPoint(r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\DoryNPATable.csv", r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\Lab3.gdb\DoryNPATable_XYTableToPoint", "Long", "Lat", None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')

<Result 'C:\\Users\\Alexander Danielson\\Desktop\\Fall 2022Spring2023\\ArcGIS I\\Lab3\\Lab3\\Lab3.gdb\\DoryNPATable_XYTableToPoint'>

## Form Optimal Route For Dory's Given Extent of Area Using Data From MNGEO and Performing Raster Analysis

## Extract County Data from MNGEO (including MN NLCD, DEM, Streams, Impervious Surfuces/Roads) to Create Optimal Route

In [3]:
CountyBoundsLink = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/bdry_counties_in_minnesota/shp_bdry_counties_in_minnesota.zip"

In [4]:
CountyBoundsLink

'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/bdry_counties_in_minnesota/shp_bdry_counties_in_minnesota.zip'

In [7]:
OutputSource = requests.post(CountyBoundsLink)

In [8]:
Sources = OutputSource.content

In [9]:
Zipp = zipfile.ZipFile(io.BytesIO(Sources))

In [10]:
Zipp.extractall(r'C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3')

In [11]:
CountyNLCDLink = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2019/tif_biota_landcover_nlcd_mn_2019.zip"

In [12]:
CountyNLCDLink

'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2019/tif_biota_landcover_nlcd_mn_2019.zip'

In [13]:
OutputNLCDSource = requests.post(CountyNLCDLink)

In [14]:
SourcesNLCD = OutputNLCDSource.content

In [15]:
ZippNLCD = zipfile.ZipFile(io.BytesIO(SourcesNLCD))

In [16]:
ZippNLCD.extractall(r'C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3')

In [17]:
CountyDEMLink = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip'

In [18]:
CountyDEMLink 

'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip'

In [19]:
OutputDEMSource = requests.post(CountyDEMLink)

In [20]:
SourcesDEM = OutputDEMSource.content

In [21]:
ZippDEM = zipfile.ZipFile(io.BytesIO(SourcesDEM))

In [22]:
ZippDEM.extractall(r'C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3')

In [23]:
CountyStreamLink = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/water_strahler_stream_order/shp_water_strahler_stream_order.zip'

In [24]:
CountyStreamLink

'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/water_strahler_stream_order/shp_water_strahler_stream_order.zip'

In [25]:
OutputStreamSource = requests.post(CountyStreamLink)

In [26]:
SourcesStream = OutputStreamSource.content

In [27]:
ZippStream = zipfile.ZipFile(io.BytesIO(SourcesStream))

In [28]:
ZippStream.extractall(r'C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3')

In [29]:
MNRoads = 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/trans_roads_mndot_tis/shp_trans_roads_mndot_tis.zip'

In [30]:
MNRoads

'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/trans_roads_mndot_tis/shp_trans_roads_mndot_tis.zip'

In [31]:
OutputMNRoads = requests.post(MNRoads)

In [32]:
SourcesMNRoads = OutputMNRoads.content

In [33]:
ZipRoads = zipfile.ZipFile(io.BytesIO(SourcesMNRoads))

In [34]:
ZipRoads.extractall(r'C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3')

## Dissolve Optimal Counties and Geoprocess Other Ancillary Data Then Transform Features to Reclassify for Weights 

### Dissolve Operation for Optimal Study Area

In [35]:
arcpy.management.Dissolve("mn_county_boundaries selection", r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\Lab3.gdb\DoryNPAStudyArea", None, None, "MULTI_PART", "DISSOLVE_LINES", '')

<Result 'C:\\Users\\Alexander Danielson\\Desktop\\Fall 2022Spring2023\\ArcGIS I\\Lab3\\Lab3\\Lab3.gdb\\DoryNPAStudyArea'>

### Extract by Mask (NLCD) using Optimal Study Area

In [36]:
out_raster = arcpy.sa.ExtractByMask("NLCD_2019_Land_Cover.tif", "DoryNPAStudyArea", "INSIDE", '524966.6376 4853462.8394 637916.1448 4922619.9426 PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'); out_raster.save(r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\Lab3.gdb\Extract_NLCD")

### Clip (Roads) using Optimal Study Area

In [39]:
arcpy.analysis.Clip("STREETS_LOAD", "DoryNPAStudyArea", r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\Lab3.gdb\StreetsClip", None)

<Result 'C:\\Users\\Alexander Danielson\\Desktop\\Fall 2022Spring2023\\ArcGIS I\\Lab3\\Lab3\\Lab3.gdb\\StreetsClip'>

### Extract by Mask (DEM) using Optimal Study Area

In [37]:
out_raster = arcpy.sa.ExtractByMask("digital_elevation_model_30m", "DoryNPAStudyArea", "INSIDE", '524966.6376 4853462.8394 637916.1448 4922619.9426 PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'); out_raster.save(r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\Lab3.gdb\Extract_DEM")

### Clip (Streams) using Optimal Study Area

In [38]:
arcpy.analysis.Clip("streams_with_strahler_stream_order", "DoryNPAStudyArea", r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\Lab3.gdb\StreamsClip", None)

<Result 'C:\\Users\\Alexander Danielson\\Desktop\\Fall 2022Spring2023\\ArcGIS I\\Lab3\\Lab3\\Lab3.gdb\\StreamsClip'>

### Create Slope out of DEM 

In [40]:
arcpy.ddd.Slope("DEMArea", r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\Lab3.gdb\SlopeDEM", "PERCENT_RISE", 1, "PLANAR", "METER")

<Result 'C:\\Users\\Alexander Danielson\\Desktop\\Fall 2022Spring2023\\ArcGIS I\\Lab3\\Lab3\\Lab3.gdb\\SlopeDEM'>

### Buffer Road Features 100 Meters

In [42]:
arcpy.analysis.Buffer("StreetsClip", r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\Lab3.gdb\StreetsClip_Buffer100M", "100 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'C:\\Users\\Alexander Danielson\\Desktop\\Fall 2022Spring2023\\ArcGIS I\\Lab3\\Lab3\\Lab3.gdb\\StreetsClip_Buffer100M'>

### Erase Riverways Where Intersection of 100 Meter Road Buffers Concide

In [ ]:
arcpy.analysis.Erase("StreamsClip", "StreetsClip_Buffer100M", r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\Lab3.gdb\StreetsClip_Buffer100M_Erase", None)

### Buffer Clipped River Feature and Convert to Raster for being Recognized in Weights then Use in Raster Calculator (In following steps)

In [43]:
arcpy.analysis.Buffer("StreetsClip_Buffer100M_Erase", r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\Lab3.gdb\StreetsClip_Buffer100_Buffer", "10 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'C:\\Users\\Alexander Danielson\\Desktop\\Fall 2022Spring2023\\ArcGIS I\\Lab3\\Lab3\\Lab3.gdb\\StreetsClip_Buffer100_Buffer'>

In [44]:
arcpy.conversion.FeatureToRaster("StreetsClip_Buffer100M_Erase", "FW_ID", r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\Lab3.gdb\BufferedRiversVector", 276.583641600005)

<Result 'C:\\Users\\Alexander Danielson\\Desktop\\Fall 2022Spring2023\\ArcGIS I\\Lab3\\Lab3\\Lab3.gdb\\BufferedRiversVector'>

## Reclassifcation

### Relcassify NLCD for Land Use

In [2]:
arcpy.ddd.Reclassify("NLCDArea", "Value", "11 15.200000 1;15.200000 19.400000 2;19.400000 23.600000 3;23.600000 27.800000 4;27.800000 32 5;32 36.200000 6;36.200000 40.400000 7;40.400000 44.600000 8;44.600000 48.800000 9;48.800000 53 10;53 57.200000 11;57.200000 61.400000 12;61.400000 65.600000 13;65.600000 69.800000 14;69.800000 74 15;74 78.200000 16;78.200000 82.400000 17;82.400000 86.600000 18;86.600000 90.800000 19;90.800000 95 20", r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\Lab3.gdb\Reclass_NLCD20classes", "DATA")

<Result 'C:\\Users\\Alexander Danielson\\Desktop\\Fall 2022Spring2023\\ArcGIS I\\Lab3\\Lab3\\Lab3.gdb\\Reclass_NLCD20classes'>

### Relcassify Slope

In [1]:
arcpy.ddd.Reclassify("SlopeDEM", "VALUE", "0 26.673990 1;26.673990 53.347980 2;53.347980 80.021970 3;80.021970 106.695959 4;106.695959 133.369949 5;133.369949 160.043939 6;160.043939 186.717929 7;186.717929 213.391919 8;213.391919 240.065909 9;240.065909 266.739899 10;266.739899 293.413889 11;293.413889 320.087878 12;320.087878 346.761868 13;346.761868 373.435858 14;373.435858 400.109848 15;400.109848 426.783838 16;426.783838 453.457828 17;453.457828 480.131818 18;480.131818 506.805807 19;506.805807 533.479797 20", r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\Lab3.gdb\Reclass_Slope20classes", "DATA")

<Result 'C:\\Users\\Alexander Danielson\\Desktop\\Fall 2022Spring2023\\ArcGIS I\\Lab3\\Lab3\\Lab3.gdb\\Reclass_Slope20classes'>

### Reclassify Streams for Study Area 1 = Rivers and 0 = Study Area

In [49]:
output_raster = arcpy.ia.RasterCalculator(' Con(IsNull( "BufferedRiversVector"),0, "BufferedRiversVector")'); output_raster.save(r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\Lab3.gdb\RasterCalStreams01")

TypeError: RasterCalculator() missing 2 required positional arguments: 'input_names' and 'expression'

# Create Buffers For Dory's Farm and North Picnic Area and Seperate Buffers

## Create Buffers For Dory's Farm and North Picnic Area and  Seperate Buffers into Own Feature Class then Convert to Raster

In [5]:
arcpy.analysis.Buffer("DoryNPATable_XYTableToPoint", r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\Lab3.gdb\DoryNPATable_XYTableT_Buffer", "100 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'C:\\Users\\Alexander Danielson\\Desktop\\Fall 2022Spring2023\\ArcGIS I\\Lab3\\Lab3\\Lab3.gdb\\DoryNPATable_XYTableT_Buffer'>

In [8]:
arcpy.conversion.FeatureToRaster("DorysFarm", "OBJECTID", r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\Lab3.gdb\DoryBuffersRaster", 0.000299576288000026)

<Result 'C:\\Users\\Alexander Danielson\\Desktop\\Fall 2022Spring2023\\ArcGIS I\\Lab3\\Lab3\\Lab3.gdb\\DoryBuffersRaster'>

In [9]:
arcpy.conversion.FeatureToRaster("NPA", "OBJECTID", r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\Lab3.gdb\NPABuffersRaster", 0.000299576288000026)

<Result 'C:\\Users\\Alexander Danielson\\Desktop\\Fall 2022Spring2023\\ArcGIS I\\Lab3\\Lab3\\Lab3.gdb\\NPABuffersRaster'>

# Input Reclassifed Rasters to Be Weighted and Create Cost Surface Analysis

## Create Weighted Raster

In [11]:
out_raster = arcpy.ia.WeightedSum("Reclass_NLCD20classes Value 1;ReclassDEM Value 3"); out_raster.save(r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\Lab3.gdb\Weighte_Recl3DEMHighNLCD20Class")

### Use Raster Calculator For Multiplying All Rasters Together (Landcover, Roads, and Streams)

In [106]:
output_raster = arcpy.ia.RasterCalculator('StreamOutput =  Con(IsNull("ReClassStreams"), 0 , "ReClassStreams" )'); output_raster.save(r"C:\Users\Alexander Danielson\Desktop\Fall 2022Springoutput_raster = arcpy.ia.RasterCalculator('StreamOutput =  Con(IsNull("ReClassStreams"), 0 , "ReClassStreams" )'); output_raster.save(r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab2\Lab2\Lab2.gdb\RasterCalculatorStreams")2023\ArcGIS I\Lab2\Lab2\Lab2.gdb\RasterCalculatorStreams")

SyntaxError: invalid syntax (<string>, line 1)

## Create Cost Surface (Distance) Raster

In [12]:
out_distance_raster = arcpy.sa.CostDistance("NPA", "Weighte_Recl3DEMHighNLCD20Class", None, None, None, None, None, None, ''); out_distance_raster.save(r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\Lab3.gdb\CostDis_NLCDHigher4")

## Create Back Link 

In [13]:
out_backlink_raster = arcpy.sa.CostBackLink("NPA", "Weighte_Recl3DEMHighNLCD20Class", None, None, None, None, None, None, ''); out_backlink_raster.save(r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\Lab3.gdb\CostBac_NLCD4")

## Create Cost Surface Allocation Raster

In [110]:
out_allocation_raster = arcpy.sa.CostAllocation("DisplayXYPathDoryFarmtoNPA", "WeightedRaster", None, None, "OBJECTID", None, None, None, None, None, None, ''); out_allocation_raster.save(r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab2\Lab2\Lab2.gdb\CosrSurfaceAllo")

## Create Cost Path for Final Route

In [14]:
out_raster = arcpy.sa.CostPath("NPA", "out_distance_raster4", "out_backlink_raster4", "EACH_CELL", "ORIG_FID", "INPUT_RANGE"); out_raster.save(r"C:\Users\Alexander Danielson\Desktop\Fall 2022Spring2023\ArcGIS I\Lab3\Lab3\Lab3.gdb\CostPat_4")